Script to extract GloFAS reforecast data at station locations stored in an s3 bucket. Metadata file is used to identify which station points to extract (use Lisflood x and y coordinates if available).

In [1]:
import s3fs
import dask
import xarray as xr
import pandas as pd
import os
from tqdm import tqdm

In [2]:
country = 'mozambique'  # define country of interest
directory = '/s3/scratch/jamie.towner/flood_aa'  # define main working directory

In [3]:
# Set up the S3 path for the Zarr files
store = f"s3://wfp-seasmon/input/cds/glofas-reforecast/saf/*/*.zarr"

# Set up connection to s3 store
s3 = s3fs.S3FileSystem.current()

# Define mapper object for multiple files
remote_files = s3.glob(store)
store = [s3fs.S3Map(root=f"s3://{file}", s3=s3, check=False) for file in remote_files]

# Configure dask to avoid creating large chunks and open the dataset
with dask.config.set(**{"array.slicing.split_large_chunks": True}):
    ds = xr.open_mfdataset(store, decode_coords="all", engine="zarr")

In [6]:
# Load the CSV file containing station information (i.e., station name, lat, lon)
# define paths to data
metadata_directory = os.path.join(directory, country, "data/metadata")
station_info_file = "metadata_observations.csv"
station_info_path = os.path.join(metadata_directory, station_info_file)
station_info = pd.read_csv(station_info_path)

# Define the output directory
out_dir = "./GloFAS_forecasts_moz"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

# Initialize tqdm progress bar
pbar = tqdm(total=len(station_info), desc="Extracting Data")

# Loop over each station in the metadata file
for index, row in station_info.iterrows():
    point_name = row['station name']
    latitude = row['lisflood_y']
    longitude = row['lisflood_x']

    # Sanitize the station name for use in a file path (e.g., remove spaces, special characters)
    station_name = "".join(c for c in point_name if c.isalnum() or c in (' ', '_')).replace(' ', '_')

    # Find the nearest latitude and longitude in the dataset
    lat_index = ds['latitude'].sel(latitude=latitude, method='nearest')
    lon_index = ds['longitude'].sel(longitude=longitude, method='nearest')

    # Extract river discharge data for the nearest point and for all ensemble members
    data_at_point = ds['dis24'].sel(latitude=lat_index, longitude=lon_index)

    # Print the actual dimensions for diagnostics
    print("Data at point dimensions:", data_at_point.dims)
    print("Data at point shape:", data_at_point.shape)

    # Extract dates, ensemble member, and step
    dates = ds.time.values
    ensemble_members = ds['dis24'].coords['number'].values
    steps = ds.step.values

    # Create a new xarray dataset for storing data in NetCDF format
    station_ds = xr.Dataset(
        {
            "dis24": (["time", "number", "step"], data_at_point.values)
        },
        coords={
            "time": dates,
            "number": ensemble_members,
            "step": steps
        },
        attrs={
            "description": f"River discharge forecasts for {point_name}",
            "units": "m^3/s"
        }
    )

    # Save the dataset as a NetCDF file
    nc_file_name = os.path.join(out_dir, f"{station_name}_forecast.nc")
    station_ds.to_netcdf(nc_file_name)

    # Update the progress bar
    pbar.update(1)

# Close the progress bar
pbar.close()

Extracting Data:   0%|          | 0/10 [00:00<?, ?it/s]

Data at point dimensions: ('number', 'time', 'step')
Data at point shape: (11, 1052, 46)


Extracting Data:  10%|█         | 1/10 [29:20<4:24:01, 1760.13s/it]

Data at point dimensions: ('number', 'time', 'step')
Data at point shape: (11, 1052, 46)


KeyboardInterrupt: 